<a href="https://colab.research.google.com/github/sarikasea/Agentic-RAG-with-Llamaindex/blob/main/utilsgemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from llama_index.core import Settings
from llama_index.llms import GoogleGenerativeAI
from llama_index.embeddings import GoogleGenerativeAIEmbedding
from llama_index.core import SimpleDirectoryReader, SummaryIndex, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from google.colab import userdata

def get_router_query_engine(file_path, llm=None, embed_model=None):
    """Get router query engine."""

    # set Google API key
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

    if llm is None:
        llm = GoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.1)
    if embed_model is None:
        embed_model = GoogleGenerativeAIEmbedding(model="models/embedding-001")

    Settings.llm = llm
    Settings.embed_model = embed_model

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)

    summary_index = SummaryIndex(nodes)
    vector_index = VectorStoreIndex(nodes)

    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize", use_async=True
    )
    vector_query_engine = vector_index.as_query_engine()

    summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description="Useful for summarization questions related to MetaGPT",
    )

    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description="Useful for retrieving specific context from the MetaGPT paper.",
    )

    query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[
            summary_tool,
            vector_tool,
        ],
        verbose=True,
    )

    return query_engine